In [ ]:
from src.data_utils.dataset import build_datset_train_val
from src.prod_models.builder import build_prod_transformer_model_v1
from src.callbacks import get_predefine_callbacks
import optuna
import tensorflow as tf
from src.constants import TARGET_MAX_LENGHT, MAX_LENGHT_SOURCE

TRIALS=3
EPOCHS=5000
EPOCHS_PER_TRIAL=15
BATCH_SIZE = 128
TRAIN_SPLIT = 0.8

In [ ]:
train_dataset, val_dataset = build_datset_train_val(split=TRAIN_SPLIT, batch_size=BATCH_SIZE)

In [ ]:
def objective(trial):
    tf.keras.backend.clear_session()

    model = build_prod_transformer_model_v1(trial=trial)
    model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS_PER_TRIAL, callbacks=get_predefine_callbacks(model_name="prod_v1"))

    levenshtein = model.evaluate(val_dataset)[-1]
    # val_loss = model.evaluate(val_dataset)[0]
    # return  val_loss
    return  levenshtein

In [ ]:
study = optuna.create_study(direction='minimize', storage="sqlite:///db.sqlite3", study_name="prod_v1")
study.optimize(objective, n_trials=TRIALS, gc_after_trial=True, show_progress_bar=True)

In [ ]:
tf.keras.backend.clear_session()
trials = study.best_trials

for index, trial in enumerate(trials):
    print(f"Best model: {index+1}")
    model_name = "prod_v1"

    model = build_prod_transformer_model_v1(trial=trial)
    model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS, callbacks=get_predefine_callbacks(model_name=model_name))
    print(model.summary())

    print('validation levenshtein distance: {}'.format(trial.value))
    print("Best hyperparameters: {}".format(trial.params))

    model.load_weights(f"../best_model/prototype/{model_name}")

    print(f"Metrics in Validation: {model.evaluate(val_dataset)}")

In [ ]:
# save model

model.save("../models/prod_v1.keras", save_format="keras")

In [ ]:
from src.data_utils.dataset import char_to_num, num_to_char

In [ ]:
target_sequence = [char_to_num[w] for w in ["<"]]

for batch_index, batch in enumerate(val_dataset):
    batch = batch[0]

    sources = batch["source"]
    targets = batch["target"]
    
    print(sources.shape)
    print(targets.shape)

    for index_sample, (source, target) in enumerate(zip(sources, targets)):
        source = tf.expand_dims(source, axis=0)
        target_sequence = [char_to_num[w] for w in ["<"]]
        y_true = "".join([num_to_char[w] for w in target.numpy()])
    
        for i in range(TARGET_MAX_LENGHT):
            next_token = tf.expand_dims(tf.pad(tf.constant(target_sequence), [[0, TARGET_MAX_LENGHT-len(target_sequence)]], mode='CONSTANT', constant_values=0, name=None), axis=0)

            print("next target sequence to predict: ", next_token)

            y_pred = model({"source": source, "target": next_token})

            y_pred = tf.cast(tf.argmax(y_pred, axis=2), dtype=tf.int32)

            print("argmax:", y_pred)

            mask = tf.not_equal(y_pred, 0)
            next_token = y_pred[mask][-1].numpy()

            target_sequence.append(next_token)

            print("sequence so far: ", "".join([num_to_char[w] for w in target_sequence]))
            print("Label: ", y_true)

            if num_to_char[next_token]==">":
                break

        print(f"================================={index_sample}=========================================")
        if index_sample==1:
            break

    if batch_index==1:
        break

In [ ]:
from src.custom.metrics import SparseLevenshtein

model_loaded = tf.keras.models.load_model("../models/prod_v1.keras", custom_objects={"SparseLevenshtein": SparseLevenshtein})

In [ ]:
#after register class as serializable
model_loaded = tf.keras.models.load_model("../models/prod_v1.keras")